In [2]:
import sys
import os

# Add root directory of rhe project into sys.path
sys.path.append("/kaggle/input/license-plate-module")

## **0. Libraries**

In [3]:
from src.visualization import display_images_and_targets
from src.data_preprocessing import download_dataset, preprocess_data
from src.dataset import LicensePlateDataset
from src.model import FTFasterRCNN
from src.train import train_model
from src.evaluate import evaluate_model
from src.callbacks import EarlyStopping, ModelCheckpoints
from src.config import (
    RAW_DATA_DIR,
    PROCESSED_DATA_DIR, 
    IMAGE_SIZE, 
    MEAN_NORMALIZATION, 
    STD_NORMALIZATION,
    NUM_WORKERS,
    BATCH_SIZE,
    NUM_CLASSES,
    DEVICE,
    EPOCHES,
    LEARNING_RATE,
    WEIGHT_DECAY,
    MOMENTUM,
    STEP_SIZE,
    GAMMA
)

from torchvision import transforms
from torch.utils.data import DataLoader
from torch.optim import SGD
from torch.optim.lr_scheduler import StepLR

import torchinfo

import warnings
warnings.filterwarnings("ignore")

## **1. Dataset**

In [4]:
# Download dataset from Github
download_dataset(dest=RAW_DATA_DIR)

Dataset does not exist, please waiting to download data from the cloud...
Start to download...
0/20 of data are downloaded
5/20 of data are downloaded
10/20 of data are downloaded
15/20 of data are downloaded


license-plate-project.zip: 100%|██████████| 965M/965M [00:10<00:00, 95.3MB/s] 


Downloaded, please waiting to extract file...
Extracting...
Extracted!!
Removing zip file...
Removed!!
Done!!


In [5]:
# Data preprocessing
preprocess_data(RAW_DATA_DIR, PROCESSED_DATA_DIR, IMAGE_SIZE)

Preprocessing data in train folder...: 100%|██████████| 20580/20580 [00:53<00:00, 385.92it/s]


In [6]:
# Get the list of file paths in each folder
file_pathes = {}
for dir_name in os.listdir(PROCESSED_DATA_DIR):
    dir_path = os.path.join(PROCESSED_DATA_DIR, dir_name)
    file_pathes[dir_name] = []
    for filename in os.listdir(dir_path):
        file_pathes[dir_name].append(os.path.join(dir_path, filename))

In [7]:
def collate_fn(batch):
    """
    Custom collate function to handle batches with varying number of bounding boxes.
    """
    images, targets = zip(*batch)
    images = list(images)  # Keep images as a list
    targets = list(targets)  # Keep targets as a list of dictionaries
    return images, targets

In [8]:
# Transform images
transform = transforms.Compose([
    transforms.Resize(IMAGE_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(mean=MEAN_NORMALIZATION, std=STD_NORMALIZATION)
])

# Declare dataset and dataloader
train_dataset = LicensePlateDataset(file_paths=file_pathes["train"], transform=transform)
valid_dataset = LicensePlateDataset(file_paths=file_pathes["valid"], transform=transform)
test_dataset = LicensePlateDataset(file_paths=file_pathes["test"], transform=transform)

# Split dataset to batches
train_dataloader = DataLoader(
    dataset=train_dataset,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    persistent_workers=True, 
    pin_memory=True,
    shuffle=True,
    prefetch_factor=10,
    collate_fn=collate_fn
)

valid_dataloader = DataLoader(
    dataset=valid_dataset,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    persistent_workers=True, 
    pin_memory=True,
    shuffle=True,
    prefetch_factor=10,
    collate_fn=collate_fn
)

test_dataloader = DataLoader(
    dataset=test_dataset,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    persistent_workers=True, 
    pin_memory=True,
    shuffle=True,
    prefetch_factor=10,
    collate_fn=collate_fn
)

## **2. Faster RCNN model**

In [9]:
faster_rcnn_model = FTFasterRCNN(num_classes=NUM_CLASSES, freeze_backbone=True)
faster_rcnn_model.to(DEVICE)
torchinfo.summary(faster_rcnn_model, (3, IMAGE_SIZE[0], IMAGE_SIZE[1]), batch_dim=0, depth=3)

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:06<00:00, 15.0MB/s]


Layer (type:depth-idx)                                  Output Shape              Param #
FTFasterRCNN                                            [1, 4]                    --
├─FasterRCNN: 1-1                                       [1, 4]                    --
│    └─GeneralizedRCNNTransform: 2-1                    [1, 3, 800, 800]          --
│    └─Sequential: 2-2                                  [1, 1024, 50, 50]         --
│    │    └─Conv2d: 3-1                                 [1, 64, 400, 400]         (9,408)
│    │    └─BatchNorm2d: 3-2                            [1, 64, 400, 400]         (128)
│    │    └─ReLU: 3-3                                   [1, 64, 400, 400]         --
│    │    └─MaxPool2d: 3-4                              [1, 64, 200, 200]         --
│    │    └─Sequential: 3-5                             [1, 256, 200, 200]        (215,808)
│    │    └─Sequential: 3-6                             [1, 512, 100, 100]        (1,219,584)
│    │    └─Sequential: 3-7         

In [10]:
optimizer = SGD(
    filter(lambda p: p.requires_grad, faster_rcnn_model.parameters()),
    lr=LEARNING_RATE, 
    momentum=MOMENTUM,
    weight_decay=WEIGHT_DECAY
)
lr_scheduler = StepLR(
        optimizer=optimizer, 
        step_size=STEP_SIZE,
        gamma=GAMMA
    )
early_stopping = EarlyStopping(patience=10)
model_checkpoints = ModelCheckpoints("faster_rcnn_checkpoints.pt")

In [ ]:
faster_rcnn_history = train_model(
                        model=faster_rcnn_model, 
                        train_dataloader=train_dataloader,
                        epoches=EPOCHES,
                        optimizer=optimizer,
                        lr_scheduler=lr_scheduler,
                        valid_dataloader=valid_dataloader,
                        early_stopping=early_stopping,
                        model_checkpoint=model_checkpoints
                )

Epoch 1/30: 100%|██████████| 1286/1286 [22:54<00:00,  1.07s/it, loss=0.286]


Validation set: Average loss: 0.2448 - Accuracy: 2854/9018 (31.65%)



Epoch 2/30: 100%|██████████| 1286/1286 [20:09<00:00,  1.06it/s, loss=0.257]


Validation set: Average loss: 0.2021 - Accuracy: 2286/6202 (36.86%)



Epoch 3/30: 100%|██████████| 1286/1286 [20:53<00:00,  1.03it/s, loss=0.221]


...

Epoch 8/30: 100%|██████████| 1286/1286 [22:54<00:00,  1.07s/it, loss=0.162]


Validation set: Average loss: 0.1374 - Accuracy: 2048/3458 (59.22%)



Epoch 9/30: 100%|██████████| 1286/1286 [23:08<00:00,  1.08s/it, loss=0.156]


Validation set: Average loss: 0.1447 - Accuracy: 2043/3495 (58.45%)



Epoch 10/30:  63%|██████▎   | 812/1286 [14:40<08:23,  1.06s/it, loss=0.153]

In [ ]:
#test_images, test_targets, test_predictions = evaluate_model("/kaggle/input/faster-rcnn/pytorch/license-plate-faster-rcnn/1/best_faster_rcnn_checkpoints.pt", test_dataloader)

In [ ]:
#display_images_and_targets(test_images, test_predictions)